<a href="https://colab.research.google.com/github/sakshirikhe0192/ML/blob/master/Final_SalePrice_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#importing dataframe
import pandas as pd

train=pd.read_csv("train.csv")

In [0]:
#seprating numerical and categorical dataframe
train_numerical=train.select_dtypes(include=['int64',]).copy()
train_numerical_col=train_numerical.columns

train_categorical=train.select_dtypes(include=['object']).copy()
train_categorical_col=train_categorical.columns

In [0]:
#No null in numerical
train_categorical.fillna('NA',inplace=True)

In [0]:
#finding skewness and taking log for numerical data
import numpy as np

class skew_dict(dict):
  def __init__(self): 
        self = dict() 

  def add(self,key,value):
    self[key]=value

skewd=skew_dict()
#kurtd=skew_dict()
for i in train_numerical_col:
        skewd.add(i,train_numerical[i].skew())
        #kurtd.add(i,train_numerical[i].kurt())

for j in skewd:
  if (skewd[j] < -0.5) or (skewd[j] > 0.5):
    print(j)
    print('{0}_Log'.format(j))
    train_numerical['{0}_Log'.format(j)]=np.log(train_numerical[j])
    train_numerical.drop(j,inplace=True,axis=1)

train_numerical.replace([-np.inf],0,inplace = True)
train_numerical

In [0]:
#Applying label encoding on categorical data
from sklearn.preprocessing import LabelEncoder
lb_make=LabelEncoder()

for i in train_categorical_col:
  train_categorical[i]=lb_make.fit_transform(train_categorical[i])
train_categorical

In [0]:
#finding skewness and taking log for categorical data
import numpy as np

class skew_dict(dict):
  def __init__(self): 
        self = dict() 

  def add(self,key,value):
    self[key]=value

skewd=skew_dict()
#kurtd=skew_dict()
for i in train_categorical_col:
        skewd.add(i,train_categorical[i].skew())
        #kurtd.add(i,train_numerical[i].kurt())
print(skewd)

for j in skewd:
  if (skewd[j] < -0.5) or (skewd[j] > 0.5):
    print(j)
    print('{0}_Log'.format(j))
    train_categorical['{0}_Log'.format(j)]=np.log(train_categorical[j])
    train_categorical.drop(j,inplace=True,axis=1)
#Replacing nan and -infs with 0

train_categorical.replace([-np.inf],0,inplace = True)
train_categorical

In [0]:
#replacing column name with transforemd log column name

train_numerical_col=train_numerical.columns
train_categorical_col=train_categorical.columns

In [0]:
# taking SalePrice in 1 dataframe for both categorical and numerical plotting
SalePrice_Log_df=pd.DataFrame()
SalePrice_Log_df['SalePrice_Log']=np.log(train['SalePrice']).copy()
SalePrice_Log_df

In [0]:
#plotting corr for numerical and categorical

from scipy.stats.stats import pearsonr
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
train_numerical_corr=pd.DataFrame()
nr=18
nc=2
fig, axes = plt.subplots(nr,nc, figsize=(10,50))
#train_numerical_col=train_numerical.columns

for r in range(0,18,1):
  for c in range (0,2,1):
      i=r*nc+c
      try:
        corr, p_value = pearsonr(train_numerical[train_numerical_col[i]], y=SalePrice_Log_df['SalePrice_Log'])
        sns.regplot(x=train_numerical[train_numerical_col[i]], y=SalePrice_Log_df['SalePrice_Log'],ax=axes[r][c])
        if (corr < -0.6) or (corr > 0.6):
          train_numerical_corr[train_numerical_col[i]]=train_numerical[train_numerical_col[i]].copy()
        axes[r][c].set_xlabel(train_numerical_col[i],labelpad=3,fontsize=15)
        axes[r][c].set_ylabel('SalePrice_Log',fontsize=20)
        axes[r][c].set_title('corr:{0:4.2f}'.format(corr),size=20)
        
      except IndexError as error:
        print("Running!!!") 
fig.tight_layout()
plt.savefig('correlation.jpg')
plt.show()

In [0]:
from scipy.stats.stats import pearsonr
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
train_categorical_corr=pd.DataFrame()
nr=22
nc=2
fig, axes = plt.subplots(nr,nc, figsize=(10,50))
#train_numerical_col=train_numerical.columns
count=1
for r in range(0,22,1):
  for c in range (0,2,1):
      i=r*nc+c
      try:
        corr, p_value = pearsonr(train_categorical[train_categorical_col[i]], y=SalePrice_Log_df['SalePrice_Log'])
        sns.regplot(x=train_categorical[train_categorical_col[i]], y=SalePrice_Log_df['SalePrice_Log'],ax=axes[r][c])
        #print('Yes!!',count, train_categorical_col[i], corr)
        if (corr < -0.4) or (corr > 0.4):
          #print('inside corr condition',train_categorical_col[i], corr)
          train_categorical_corr[train_categorical_col[i]]=train_categorical[train_categorical_col[i]]
        axes[r][c].set_xlabel(train_categorical_col[i],labelpad=3,fontsize=15)
        axes[r][c].set_ylabel('SalePrice_Log',fontsize=20)
        axes[r][c].set_title('corr:{0:4.2f}'.format(corr),size=20)
        count=count+1
      except IndexError as error:
        print("Running!!!") 
fig.tight_layout()
plt.savefig('correlation.jpg')
plt.show()

In [0]:
train_categorical_corr['GarageType_Log'].dtype

In [0]:
train_numerical_corr

In [0]:
#concatenating both dataframe
train_cat_num=pd.DataFrame()
train_cat_num=pd.concat([train_numerical_corr,train_categorical_corr],axis=1)
train_cat_num

In [0]:
corr_heatmap=train_cat_num.corr()
corr_heatmap

In [0]:
for i in corr_heatmap:
  for j in corr_heatmap[i]:
     if (j>-0.55) and (j<0.6):
       corr_heatmap.replace(to_replace =j,value= np.nan,inplace=True) 

fig, ax = plt.subplots(figsize=(15,7)) 
sns.heatmap(corr_heatmap, annot = True,linewidths=.5)


In [0]:
#'OverallQual','GarageCars','ExterQual_Log'
[train_cat_num.drop(col ,axis=1,inplace=True) for col in train_cat_num if col not in ['OverallQual','GarageCars','ExterQual_Log','SalePrice_Log']]


In [0]:
train_cat_num


In [0]:
from sklearn.model_selection import train_test_split

X = train_cat_num[['OverallQual','GarageCars','ExterQual_Log']]
y = train_cat_num['SalePrice_Log']

print(X)
print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [0]:
import seaborn as seabornInstance

plt.figure(figsize=(15,10))
plt.tight_layout()
seabornInstance.distplot(train_cat_num['SalePrice_Log'])

In [0]:
regressor = LinearRegression()  
regressor.fit(X_train, y_train)

In [0]:
#To see what coefficients our regression model has choosen
from sklearn.linear_model import LinearRegression

coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])  
coeff_df

In [0]:
y_pred = regressor.predict(X_test)

In [0]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

df.head(25)

In [0]:
#plot the comparison of Actual and Predicted values

df1.plot(kind='bar',figsize=(10,8))
#plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
#plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [0]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))